# 🎯 Exercise: Build an Image Generation Agent with Cost Approval

Build an agent that generates images using the MCP server, but requires approval for "bulk" image generation:

- Single image request (1 image): Auto-approve, generate immediately
- Bulk request (>1 image): Pause and ask for approval before generating multiple images
- Explore different publicly available Image Generation MCP Servers

### Import Necessary Libraries

In [1]:
import uuid
import asyncio
import requests
import base64
from datetime import datetime
from pathlib import Path
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

from PIL import Image
import io
import base64
from io import BytesIO

try:
    from PIL import Image, ImageDraw, ImageFont
    PIL_AVAILABLE = True
except ImportError:
    PIL_AVAILABLE = False
    print("⚠️  PIL not available. Install with: pip install pillow")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Loading Environment Variables

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

# Loading environment variables
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")

In [3]:
# Retry configuration for API calls
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

### MCP integration with Image Generation Server

Requires to have a replicate account and have API key

In [4]:
# MCP integration with Everything Server
# Source: https://github.com/GongRzhe/Image-Generation-MCP-Server

mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@gongrzhe/image-gen-server",
            ],
            env={
                "REPLICATE_API_TOKEN": REPLICATE_API_TOKEN,
                "MODEL": "black-forest-labs/flux-1.1-pro"
            },
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


### Helper Functions

In [5]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                # if part.text:
                #     print(f"Agent > {part.text}")
                if hasattr(part, "function_response") and part.function_response:
                        for item in part.function_response.response.get("content", []):
                            online_image_url = item['text']
                            print(f"Image URL: {online_image_url}")
                            online_image_url = online_image_url.replace('"', '')
                            response = requests.get(online_image_url)
                            img = Image.open(BytesIO(response.content))
                            # img

def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


print("✅ Helper functions defined")

✅ Helper functions defined


In [6]:
BULK_IMAGE_THRESHOLD = 1  # More than 1 image requires approval

def generate_image_with_approval(
    num_images: int, prompt: str, tool_context: ToolContext
) -> dict:
    """Generates images with approval workflow. Requires approval if requesting more than 1 image (BULK_IMAGE_THRESHOLD).

    Args:
        num_images: Number of images to generate
        prompt: Text description for image generation
        tool_context: ADK tool context for approval workflow

    Returns:
        Dictionary with generation status and details
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small requests (≤1 image) auto-approve - generate immediately
    if num_images <= BULK_IMAGE_THRESHOLD:
        return {
            "status": "approved",
            "request_id": f"IMG-{num_images}-AUTO",
            "num_images": num_images,
            "prompt": prompt,
            "message": f"Image generation auto-approved: {num_images} image(s) with prompt '{prompt}'",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Bulk requests need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Bulk image generation: {num_images} images with prompt '{prompt}'. Do you want to approve?",
            payload={"num_images": num_images, "prompt": prompt},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Bulk generation of {num_images} images requires approval",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "request_id": f"IMG-{num_images}-HUMAN",
            "num_images": num_images,
            "prompt": prompt,
            "message": f"Image generation approved: {num_images} image(s) with prompt '{prompt}'",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Image generation rejected: {num_images} image(s) with prompt '{prompt}'",
        }
print("✅ Long-running functions created!")

✅ Long-running functions created!


In [7]:
# Create image agent with MCP integration and approval workflow
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-pro", retry_options=retry_config),
    name="image_agent",
    instruction="""IMAGE GENERATION AGENT WITH APPROVAL WORKFLOW

YOUR WORKFLOW (FOLLOW THESE STEPS IN ORDER):

STEP 1: ANALYZE USER REQUEST
- Extract the number of images requested (default to 1 if not specified)
- Extract the prompt/description for the images
- Identify key phrases: "10 images", "5 pictures", "generate 3...", etc.

STEP 2: CHECK APPROVAL STATUS
- ALWAYS call generate_image_with_approval FIRST with:
  * num_images: the number extracted from user request
  * prompt: the description for the images
- This function handles the approval workflow automatically

STEP 3: HANDLE APPROVAL RESPONSE
- If status = "approved": Proceed to generate images with MCP tool
- If status = "pending": Wait (human approval in progress, workflow will resume)
- If status = "rejected": Inform user the request was rejected

STEP 4: GENERATE IMAGES (ONLY IF APPROVED)
- Use the generate_image MCP tool to create the actual images
- Pass the prompt and num_outputs parameter
- Report results to user

IMPORTANT RULES:
• ALWAYS call generate_image_with_approval BEFORE generating any images
• Extract num_images accurately from user request (look for numbers + "images"/"pictures"/"photos")
• If user says "a cat", that means 1 image
• If user says "5 cats", that means 5 images of cats
• Don't ask confirmation yourself - the approval function handles that
• Be concise in responses - no unnecessary explanations

EXAMPLE FLOWS:

User: "Generate 1 cat image"
You: [Call generate_image_with_approval(num_images=1, prompt="cat")]
     → Status: approved (auto-approved, ≤1 image)
     [Call generate_image MCP tool]
     → "✅ Generated 1 image successfully"

User: "Create 5 sunset pictures"  
You: [Call generate_image_with_approval(num_images=5, prompt="sunset")]
     → Status: pending (requires approval, >1 image)
     [Workflow pauses for human approval]
     → If approved: [Call generate_image MCP tool]
     → "✅ Generated 5 images successfully"
""",
    tools=[mcp_image_server, FunctionTool(func=generate_image_with_approval)],
)

print("✅ Image Generation Agent created!")

✅ Image Generation Agent created!


In [8]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
image_gen_app = App(
    name="image_generator_coordinator",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

✅ Resumable app created!


/var/folders/0_/y72yvm_x0pl62q2dh2nbv91c0000gn/T/ipykernel_82707/908889127.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [9]:
session_service = InMemorySessionService()

# Create runner with the resumable app
image_gen_runner = Runner(
    app=image_gen_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

✅ Runner created!


## Image Generation Workflow - How It Works

This workflow demonstrates the **pause-and-resume** pattern for human-in-the-loop approval:

### The 3-Step Process:

**STEP 1: Initial Request**
- User sends image generation request
- Agent calls `generate_image_with_approval()` tool
- If ≤1 image: Auto-approved, continues immediately
- If >1 image: Returns `adk_request_confirmation` event and **PAUSES**

**STEP 2: Check for Approval**
- Scan events for `adk_request_confirmation`
- If present: Workflow needs human decision
- If absent: Single image request completed

**STEP 3: Resume with Decision**
- If approval needed: Call `run_async()` again with:
  - Same `session_id` (maintains conversation context)
  - Same `invocation_id` (critical - tells ADK to **RESUME**)
  - `create_approval_response()` with human decision (approve/reject)
- Agent continues from where it paused

### Key Components:

- **`generate_image_with_approval()`**: Function tool that implements 3 scenarios (auto-approve, pause, resume)
- **`check_for_approval()`**: Detects if `adk_request_confirmation` event exists
- **`create_approval_response()`**: Creates the response message to resume the workflow
- **`invocation_id`**: The "resume token" that reconnects the paused workflow

### Test Scenarios:

1. **Single image** - No approval needed (≤ threshold)
2. **Bulk with approval** - Pauses, then approves
3. **Bulk with rejection** - Pauses, then rejects


In [10]:
async def run_image_generation_workflow(query: str, auto_approve: bool = True):
    """Runs an image generation workflow with approval handling.
    
    Args:
        query: User's image generation request
        auto_approve: Whether to auto-approve bulk requests (simulates human decision)
    """
    
    print(f"\n{'='*60}")
    print(f"User > {query}\n")
    
    # Generate unique session ID
    session_id = f"img_{uuid.uuid4().hex[:8]}"
    
    # Create session
    await session_service.create_session(
        app_name="image_generator_coordinator", user_id="test_user", session_id=session_id
    )
    
    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_images > BULK_IMAGE_THRESHOLD, 
    # the Agent returns the special `adk_request_confirmation` event
    async for event in image_gen_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a bulk request - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")
        
        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in image_gen_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            # img = Image.open(BytesIO(event))
            # img
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if hasattr(part, "function_response") and part.function_response:
                        if part.function_response.response.get("content", []) != []:
                            print(f"Image URL: {part.function_response.response.get('content', [])[0]['text']}")
                    # if part.text:
                # if event.content and event.content.parts:
                #     for part in event.content.parts:
                #         if hasattr(part, "function_response") and part.function_response:
                #             for item in part.function_response.response.get("content", []):
                #                 online_image_url = item['text']
                #                 online_image_url = online_image_url.replace('"', '')
                #                 print(f"Image URL: {online_image_url}")
                                # online_response = requests.get(online_image_url)
                                # img = Image.open(BytesIO(online_response.content))
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - 
        # single image request completed immediately.
        print_agent_response(events)
    
    print(f"{'='*60}\n")

print("✅ Image generation workflow function ready")


✅ Image generation workflow function ready


### Testing with different scenarios

In [11]:
# Scenario 1: Single image request (auto-approved, no pause)
await run_image_generation_workflow("Generate a cute cat on a windowsill")


User > Generate a cute cat on a windowsill



/opt/homebrew/Caskroom/miniforge/base/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


Image URL: "https://replicate.delivery/xezq/fmPEft73HLlQRE5nG4JIqTNzm5R8WRaig1f8UqcgIVvnK1TrA/tmpnt_vwhq0.webp"



In [12]:
# Scenario 2: Bulk request with auto-approval
await run_image_generation_workflow("Create 5 beautiful sunset images", auto_approve=True)



User > Create 5 beautiful sunset images



/opt/homebrew/Caskroom/miniforge/base/lib/python3.12/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.12/site-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅



Image URL: "https://replicate.delivery/xezq/fZn48NDIaOT3dSDTfvhyBx0T9BOhN564WiJjZMD8uH3ll6pVA/tmpajrdggbq.webp"
Image URL: "https://replicate.delivery/xezq/LtUmdB0L8m6PG9rNA2nPhDwGSa90MtUvnVSeBChPb6BzS90KA/tmpsryj4n1r.webp"



In [13]:
# # Scenario 3: Bulk request with rejection
await run_image_generation_workflow("Generate 9 abstract art pieces", auto_approve=False)


User > Generate 9 abstract art pieces



⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌


